In [1]:
####### dropout resnet18 vs without dropout
#### 
import torch
import sys
sys.path.append("..")
import numpy as np

CLEAN_PATH = "/home/yifan/dataset/resnet18_with_dropout/pairflip/cifar10/0/"
REF_PATH = "/home/yifan/dataset/clean/pairflip/cifar10/0"


ENCODER_DIMS=[512,256,256,256,256,2]
DECODER_DIMS= [2,256,256,256,256,512]
VIS_MODEL_NAME = 'vis'

DEVICE='cuda:0'
########## initulize reference data and target data
from alignment.data_preprocess import DataInit
REF_EPOCH = 200
TAR_EPOCH = 200
ref_datainit = DataInit(REF_PATH,REF_PATH,REF_EPOCH)
tar_datainit = DataInit(CLEAN_PATH,CLEAN_PATH,TAR_EPOCH)
ref_model, ref_provider, ref_train_data, ref_prediction, ref_prediction_res, ref_scores = ref_datainit.getData()
tar_model, tar_provider, tar_train_data, tar_prediction, tar_prediction_res, tar_scores = tar_datainit.getData()


from alignment.ReferenceGenerator import ReferenceGenerator
gen = ReferenceGenerator(ref_provider=ref_provider, tar_provider=tar_provider,REF_EPOCH=REF_EPOCH,TAR_EPOCH=TAR_EPOCH,ref_model=ref_model,tar_model=tar_model,DEVICE=DEVICE)

absolute_alignment_indicates,predict_label_diff_indicates,predict_confidence_Diff_indicates,high_distance_indicates = gen.subsetClassify(mes_val_for_diff=18,mes_val_for_same=0.8,conf_val_for_diff=0.3,conf_val_for_same=0.05)


from representationTrans.trans_visualizer_border import visualizer
from singleVis.SingleVisualizationModel import VisModel
from singleVis.projector import TimeVisProjector
model = VisModel(ENCODER_DIMS, DECODER_DIMS)

I = np.eye(512)
projector = TimeVisProjector(vis_model=model, content_path=REF_PATH, vis_model_name=VIS_MODEL_NAME, device="cpu")
vis = visualizer(ref_provider, I,I, np.dot(ref_provider.train_representation(TAR_EPOCH),I), projector, 200,[0,1],'tab10')


/home/yifan/miniconda3/envs/deepdebugger/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NET resnet18
Finish initialization...


100%|██████████| 250/250 [00:00<00:00, 282.39it/s]


NET resnet18_with_dropout
Finish initialization...


100%|██████████| 250/250 [00:00<00:00, 5056.62it/s]


absolute alignment indicates number: 106 label diff indicates number: 12 confidence diff indicates number: 16 high distance number: 97


In [2]:
from AlignVis.autoencoder import GCNAutoencoder
import torch.optim as optim
import torch.nn as nn
X = torch.Tensor(ref_train_data)
Y = torch.Tensor(tar_train_data)

In [3]:
def train_autoencoder(X, Y, output_dim, batch_size=128):
    # Build the graph
    Y_dist = np.sqrt(np.sum((Y[:, None, :] - Y[None, :, :])**2, axis=-1))
    adj = np.exp(-Y_dist**2 / (2 * np.median(Y_dist)**2))
    adj = adj / adj.sum(axis=1, keepdims=True)
    adj = torch.tensor(adj).float()

    # Normalize the input data
    X_norm = (X - X.mean(axis=0)) / X.std(axis=0)
    Y_norm = (Y - Y.mean(axis=0)) / Y.std(axis=0)

    # Initialize the model and the optimizer
    input_dim = X_norm.shape[1]
    hidden_dim = 32
    autoencoder = GCNAutoencoder(input_dim, output_dim, hidden_dim)
    optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)
    criterion = nn.MSELoss()

    # Train the autoencoder
    for epoch in range(100):
        # Shuffle the input data
        indices = torch.randperm(Y_norm.shape[0])
        Y_norm = Y_norm[indices]

        # Process the input data in batches
        for i in range(0, Y_norm.shape[0], batch_size):
            batch = Y_norm[i:i+batch_size]
            optimizer.zero_grad()
            output_data, encoded_Y = autoencoder(batch, adj)

            # Compute the MGM loss
            Y_dist_batch = Y_dist[i:i+batch_size, i:i+batch_size]
            output_dist = np.sqrt(np.sum((encoded_Y[:, None, :] - encoded_Y[None, :, :])**2, axis=-1))
            output_dist = torch.tensor(output_dist).float()
            output_dist_batch = output_dist[i:i+batch_size, i:i+batch_size]
            loss1 = torch.mean((Y_dist_batch - output_dist_batch)**2 / (Y_dist_batch**2 + 1e-8))

            # Add GCN regularization term
            loss2 = torch.norm(torch.mm(adj, encoded_Y), p=2)

            # Total loss
            loss = loss1 + 0.01 * loss2
            loss.backward()
            optimizer.step()

    # Extract the transformed data Y_trans
    with torch.no_grad():
        _, encoded_Y = autoencoder(Y_norm, adj)
        Y_trans = encoded_Y.detach().numpy()

    # Compute the Frobenius norm between X and Y_trans
    fro_norm = torch.norm(X_norm - encoded_Y[:X_norm.shape[0], :], p='fro')

    # Compute the CKA between Y and Y_trans
    Y_norm_flat = Y_norm.flatten

In [5]:
# Set the hyperparameters
input_dim = 512
hidden_dim = 2
lr = 0.01
num_epochs = 1000

In [6]:
Y_trans, norm, cka = train_autoencoder(X, Y, input_dim)

RuntimeError: [enforce fail at alloc_cpu.cpp:66] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 5120000000000 bytes. Error code 12 (Cannot allocate memory)